In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import os

## Add new data

### Canola

Pipeline: 
* Fix excel files.
* Input LOC acronym into dict file.
* Convert ENTRY - NAME and BRAND separated by YEAR.
* Convert LOC after final dataset has been established.
* Remove numerical data that is not YIELD.
* Change dots to NaN.
* Yield is currently on lb/acre.
  
Groups:
* 2004 - 2011
* 2012 - 2021
* 2022 - 2023

#### Getting csv files based on groups

In [91]:
folder_path = 'canola/pre_process/'
path_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.xlsx')]

path_list

['canola/pre_process/2012.xlsx',
 'canola/pre_process/2013.xlsx',
 'canola/pre_process/2014.xlsx',
 'canola/pre_process/2015.xlsx',
 'canola/pre_process/2016.xlsx',
 'canola/pre_process/2017.xlsx',
 'canola/pre_process/2018.xlsx',
 'canola/pre_process/2019.xlsx',
 'canola/pre_process/2020.xlsx',
 'canola/pre_process/2021.xlsx']

In [105]:
path_list = ['canola/pre_process/2022.xlsx', 'canola/pre_process/2023.xlsx']

In [109]:
res = pd.DataFrame()
for path in path_list:
    all_sheets_dict = pd.read_excel(path, sheet_name=None)
    for sheet_name, df in all_sheets_dict.items():
        print(path, sheet_name)
        if 'YLD' in df.columns:
            # renaming name to Line
            if 'NAME' in df.columns:
                df.rename(columns={'NAME': 'Line'}, inplace=True)
            elif 'LINE' in df.columns:
                df.rename(columns={'LINE': 'Line'}, inplace=True)
            df = df[['YEAR','LOC','ENTRY','Line','CITY','STATE','YLD']]
            # concat of all locations in a year
            res = pd.concat([res, df], axis=0, ignore_index=True)

    res = res[['YEAR','ENTRY','LOC','Line','CITY','STATE','YLD']]

canola/pre_process/2022.xlsx 2022 NWCVTHYB
canola/pre_process/2022.xlsx NWCVTOP
canola/pre_process/2023.xlsx 2023 NWCVT - BOTH
canola/pre_process/2023.xlsx 2023 NWCVT HYBRID
canola/pre_process/2023.xlsx 2023 NWCVT OP


In [110]:
res = res[~res['YLD'].astype(str).str.contains('^\\.$')]
res = res.dropna()

In [111]:
res.YEAR = res.YEAR.astype(int)
res = res[res.YLD != 0.0]
res.YLD = res.YLD.astype(int)
res.ENTRY = res.ENTRY.astype(int)
res.head()
res.to_csv("canola.csv", index=False)

#### Adding loc

In [144]:
loc_df = pd.read_csv("canola/loc.csv")
loc_df.COUNTY = loc_df.COUNTY.str.upper()
loc_df 

,LOC,COUNTY,STATE
0,ALA,AUBURN,AL
1,ALM,MERIDIANVILLE,AL
2,ALN,NORMAL,AL
3,ARK,KIBLER,AR
4,ARM,MARIANNA,AR
...,...,...,...
175,TNMHYB,MEMPHIS,TN
176,TNMOP,MEMPHIS,TN
177,TXBHYB,BUSHLAND,TX
178,TXBOP,BUSHLAND,TX


In [145]:
df = pd.read_csv('canola_g2.csv')
df.LOC = df.LOC.str.replace('-', '')
df.head(-10)

,YEAR,ENTRY,LOC,LINE,YLD
0,2012,16,ALA,MH07J14,1863
1,2012,1,ALA,Rossini,1499
2,2012,3,ALA,TCI806,1006
3,2012,13,ALA,Safran,894
4,2012,12,ALA,Hornet,1888
...,...,...,...,...,...
16113,2021,6,VTA,Riley,3012
16114,2021,5,VTA,Surefire,3565
16115,2021,9,VTA,CP1055WC,1037
16116,2021,12,VTA,MH 17HID007,1417


In [146]:
res = pd.merge(df, loc_df, on='LOC', how='left')
res = res[['YEAR', 'ENTRY', 'LOC', 'STATE', 'COUNTY', 'LINE', 'YLD']]
res
res.to_csv('canola_g22.csv', index=False)

In [150]:
# joining all
g1 = pd.read_csv('canola_g1.csv')
g2 = pd.read_csv('canola_g2.csv')
g3 = pd.read_csv('canola_g3.csv')
res = pd.concat([g1, g2, g3], axis=0, ignore_index=True)
res.to_csv('canola.csv', index=False)

#### Adding NAME and BRAND based on combination YEAR + ENTRY

In [4]:
# check with 2004 and 2005 VSX-2 VSU
entries = pd.read_csv('canola/dict.csv')
df = pd.read_csv('datasets/canola.csv')

In [13]:
merged_df = pd.merge(df, entries[['YEAR', 'ENTRY', 'NAME', 'BRAND', 'TYPE']],
                     on=['YEAR', 'ENTRY'], how='left')
# Adding WATER_REGIME as unknown
merged_df['WATER_REGIME'] = 'Unknown'
merged_df = merged_df[["YEAR","LOC","STATE","COUNTY_CITY","LINE","BRAND","NAME","YIELD", "WATER_REGIME", "TYPE"]]
merged_df
merged_df.to_csv("canola.csv", index=False)

### General

In [32]:
df = pd.read_csv("1982-1992_corn.csv")
counties = pd.read_csv("datasets/county_data.csv")

In [33]:
df.drop(['PAVG'], inplace=True, axis=1)
df.YIELD = round(df.YIELD).astype(int)
df.TW = round(df.TW).astype(int)
df.MOIST = round(df.MOIST).astype(int)
df['DAYS'] = np.nan
df['STATE'] = 'KS'
df

,YEAR,LOC,BRAND,NAME,PCODE,YIELD,TW,MOIST,DAYS,STATE
0,1984,RPI,FUNK'S,G-4465,841,205,59,17,NaN,KS
1,1984,FNI,FUNK'S,G-4465,841,175,58,19,NaN,KS
2,1984,DND,FUNK'S,G-4505,842,145,57,19,NaN,KS
3,1982,DND,ASGROW,RX 90,992,127,57,17,NaN,KS
4,1984,SHI,ASGROW,RX 90,992,192,53,22,NaN,KS
...,...,...,...,...,...,...,...,...,...,...
5138,1991,RPI,CASTERLINE,CX1170,2405,159,59,13,NaN,KS
5139,1991,THI,CASTERLINE,CX1170,2405,171,63,11,NaN,KS
5140,1991,FNI,CASTERLINE,CX1170,2405,156,62,14,NaN,KS
5141,1991,DND,CASTERLINE,CX1170,2405,124,60,14,NaN,KS


In [34]:
def split_LOC(df, c_dict):
    df['COUNTY'] = df['LOC'].str[:2].map(
        c_dict
    ).fillna("Unknown")

    df['WATER_REGIME'] = df['LOC'].str[2].map(
        {'I': 'Irrigated',
         'i': 'Irrigated',
         'D': 'Dryland',
         'd': 'Dryland'
         }
    )
    return df

In [35]:
c_dict = dict(zip(counties['ACRONYM'], counties['COUNTY']))
c_dict['FN'] = 'FINNEY'
c_dict['GR'] = 'GREELEY'
c_dict['DN'] = 'DONIPHAN'
df = split_LOC(df, c_dict)
df

,YEAR,LOC,BRAND,NAME,PCODE,YIELD,TW,MOIST,DAYS,STATE,COUNTY,WATER_REGIME
0,1984,RPI,FUNK'S,G-4465,841,205,59,17,NaN,KS,REPUBLIC,Irrigated
1,1984,FNI,FUNK'S,G-4465,841,175,58,19,NaN,KS,FINNEY,Irrigated
2,1984,DND,FUNK'S,G-4505,842,145,57,19,NaN,KS,DONIPHAN,Dryland
3,1982,DND,ASGROW,RX 90,992,127,57,17,NaN,KS,DONIPHAN,Dryland
4,1984,SHI,ASGROW,RX 90,992,192,53,22,NaN,KS,SHERMAN,Irrigated
...,...,...,...,...,...,...,...,...,...,...,...,...
5138,1991,RPI,CASTERLINE,CX1170,2405,159,59,13,NaN,KS,REPUBLIC,Irrigated
5139,1991,THI,CASTERLINE,CX1170,2405,171,63,11,NaN,KS,THOMAS,Irrigated
5140,1991,FNI,CASTERLINE,CX1170,2405,156,62,14,NaN,KS,FINNEY,Irrigated
5141,1991,DND,CASTERLINE,CX1170,2405,124,60,14,NaN,KS,DONIPHAN,Dryland


In [36]:
# YEAR,LOC,STATE,COUNTY_CITY,BRAND,NAME,PCODE,YIELD,WATER_REGIME,MOIST,TW,DAYS
df.rename(columns={'COUNTY': 'COUNTY_CITY'}, inplace=True)
desired_order = ['YEAR', 'LOC', 'STATE', 'COUNTY_CITY', 'BRAND', 'NAME', 'PCODE', 'YIELD', 'WATER_REGIME', 'MOIST', 'TW', 'DAYS']
df = df[desired_order]
df

,YEAR,LOC,STATE,COUNTY_CITY,BRAND,NAME,PCODE,YIELD,WATER_REGIME,MOIST,TW,DAYS
0,1984,RPI,KS,REPUBLIC,FUNK'S,G-4465,841,205,Irrigated,17,59,NaN
1,1984,FNI,KS,FINNEY,FUNK'S,G-4465,841,175,Irrigated,19,58,NaN
2,1984,DND,KS,DONIPHAN,FUNK'S,G-4505,842,145,Dryland,19,57,NaN
3,1982,DND,KS,DONIPHAN,ASGROW,RX 90,992,127,Dryland,17,57,NaN
4,1984,SHI,KS,SHERMAN,ASGROW,RX 90,992,192,Irrigated,22,53,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5138,1991,RPI,KS,REPUBLIC,CASTERLINE,CX1170,2405,159,Irrigated,13,59,NaN
5139,1991,THI,KS,THOMAS,CASTERLINE,CX1170,2405,171,Irrigated,11,63,NaN
5140,1991,FNI,KS,FINNEY,CASTERLINE,CX1170,2405,156,Irrigated,14,62,NaN
5141,1991,DND,KS,DONIPHAN,CASTERLINE,CX1170,2405,124,Dryland,14,60,NaN


In [37]:
def is_fully_unique(df):
    fully_unique = True
    for year in df.YEAR.unique():
        is_unique_combination = (
            ~df[df.YEAR == year]
            # .duplicated(subset=["NAME", "COUNTY_CITY", "WATER_REGIME"])
            .duplicated(subset=["NAME", "COUNTY_CITY"])
            .any()
        )
        if not is_unique_combination:
            print(f"Not unique in {year}")
            fully_unique = False
    if fully_unique:
        print("Combination of YEAR, NAME, COUNTY, WATER_REGIME is now unique")

In [38]:
def drop_duplicates_by_year(df):
    grouped = df.groupby("YEAR")
    non_duplicate_df = pd.DataFrame(columns=df.columns)

    for year, group in grouped:
        duplicate_rows = group[
            # group.duplicated(subset=["NAME", "COUNTY_CITY", "WATER_REGIME"], keep=False)
            group.duplicated(subset=["NAME", "COUNTY_CITY"], keep=False)
        ]
        non_duplicate_rows = group.drop(duplicate_rows.index)
        non_duplicate_df = pd.concat([non_duplicate_df, non_duplicate_rows])

    non_duplicate_df.reset_index(drop=True, inplace=True)
    return non_duplicate_df

In [39]:
is_fully_unique(df)

Not unique in 1985
Not unique in 1986
Not unique in 1988


In [45]:
df = drop_duplicates_by_year(df)
is_fully_unique(df)
df

Combination of YEAR, NAME, COUNTY, WATER_REGIME is now unique


,YEAR,LOC,STATE,COUNTY_CITY,BRAND,NAME,PCODE,YIELD,WATER_REGIME,MOIST,TW,DAYS
0,1982,DND,KS,DONIPHAN,ASGROW,RX 90,992,127,Dryland,17,57,NaN
1,1982,DND,KS,DONIPHAN,FUNK'S,G-4507,1017,154,Dryland,17,57,NaN
2,1982,DND,KS,DONIPHAN,Northrup King,PX 74,1042,142,Dryland,17,57,NaN
3,1982,DND,KS,DONIPHAN,Northrup King,PX 95,1045,145,Dryland,22,53,NaN
4,1982,DND,KS,DONIPHAN,Dekalb,TXS 115A,1072,133,Dryland,17,56,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5128,1991,RPI,KS,REPUBLIC,CASTERLINE,CX1170,2405,159,Irrigated,13,59,NaN
5129,1991,THI,KS,THOMAS,CASTERLINE,CX1170,2405,171,Irrigated,11,63,NaN
5130,1991,FNI,KS,FINNEY,CASTERLINE,CX1170,2405,156,Irrigated,14,62,NaN
5131,1991,DND,KS,DONIPHAN,CASTERLINE,CX1170,2405,124,Dryland,14,60,NaN


In [186]:
df = df.drop_duplicates()
df.dropna(subset=["NAME"], inplace=True)

In [187]:
df

,YEAR,LOC,BRAND,NAME,PCODE,YIELD,PAVG,TW,WATER_REGIME,COUNTY
0,1982,BRD,Kansas,Triumph 64,9,40.66,106.01,58.75,Dryland,BROWN
1,1982,BRD,Kansas,Scout 66,11,35.93,93.68,56.00,Dryland,BROWN
2,1982,BRD,Nebraska,Buckskin,44,38.41,100.14,56.00,Dryland,BROWN
3,1982,BRD,Kansas,Larned,69,41.14,107.25,56.75,Dryland,BROWN
4,1982,BRD,Kansas,Newton,73,33.24,86.67,55.00,Dryland,BROWN
...,...,...,...,...,...,...,...,...,...,...
5437,1993,SUD,Ehmke Seed,Ike,495,33.50,105.65,56.35,Dryland,SUMNER
5438,1993,SUD,Kansas,(W) Arlin,501,39.80,125.52,55.92,Dryland,SUMNER
5439,1993,SUD,AgriPro,Ponderosa,504,26.70,84.21,54.25,Dryland,SUMNER
5440,1993,SUD,Kansas,Karl 92,510,33.00,104.08,55.82,Dryland,SUMNER


In [188]:
brand_elements = ["MAT CHK", "MATURITY CHECK", "zMATURITY CHECK", "MATURITY CHECK", "Z1--- EARLY CHECK", "Z4--- MED"]
name_elements = ["GRAND MEAN", "MEAN", "CV", "LSD", "Average", "AVERAGE", "LSD (0.05)", "LSD (0.05)*", "CV (%)", "LSD (0.5)", "MATURITY SHORT", "MATURITY MID", "MATURITY FULL"]
## 9903 9902 pcode 
df = df[~df['BRAND'].isin(brand_elements)]
df = df[~df['BRAND'].str.contains("CHECK", na=False)]
df = df[~df['BRAND'].str.contains("Check", na=False)]

df = df[~df['NAME'].isin(name_elements)]
df.reset_index(drop=True, inplace=True)

In [12]:
df.to_csv('sunflower_2.csv', index=False)

In [189]:
full = pd.read_csv("datasets/wheat.csv")

In [190]:
combined = pd.concat([df, full]).sort_values(by='YEAR')

In [24]:
combined = combined.drop(["CROP"], axis=1)

In [25]:
combined = combined[["YEAR","LOC","COUNTY","WATER_REGIME","BRAND","NAME","PCODE","YIELD","PAVG","MOIST","TW","DAYS"]]

In [15]:
combined.to_csv('wheat_2.csv', index=False)

## Fixing Data

### General

In [3]:
path_list = ['datasets/canola.csv','datasets/corn.csv', 'datasets/sorghum.csv', 'datasets/soybean.csv', 'datasets/sunflower.csv', 'datasets/wheat.csv']

In [5]:
df = pd.read_csv(path_list[0])
df.STATE.unique()

array(['AL', 'AR', 'VA', 'IL', 'MO', 'OH', 'KS', 'NE', 'OK', 'TX', 'IN',
       'CO', 'MT', 'WY', 'OR', 'KY', 'PA', 'GA', 'MN', 'NC', 'IA', 'NM',
       'MD', 'WA', 'NJ', 'TN', 'VT', 'MS', 'SD', 'SC', 'LA'], dtype=object)

In [157]:
# Adding State Column
for path in path_list:
    df = pd.read_csv(path)
    loc_index = df.columns.get_loc('LOC')
    df.insert(loc_index + 1, 'STATE', 'KS')
    # df.to_csv(path, index=False)

In [89]:
# Removing decimal places
for path in path_list:
    df = pd.read_csv(path)
    numerical_columns = df.select_dtypes(include='number').columns
    df[numerical_columns] = df[numerical_columns].fillna('0').round(0).astype(int)
    # df.to_csv(path, index=False)

In [92]:
# Stanton to Stafford county
for path in path_list:
    df = pd.read_csv(path)
    df.COUNTY = df.COUNTY.replace('STANTON', 'STAFFORD')
    # df.to_csv(path, index=False)

In [94]:
# FN - Finney, PH - Phillips, Greeley - GR
for path in path_list:
    df = pd.read_csv(path)
    conditions = [
    (df['LOC'].str[:2] == 'FN'),
    (df['LOC'].str[:2] == 'PH'),
    (df['LOC'].str[:2] == 'GR')
    ]
    mappings = ['FINNEY', 'PHILLIPS', 'GREELEY']
    df['COUNTY'] = np.select(conditions, mappings, default=df['COUNTY'])
    # df.to_csv(path, index=False)

### Corn

In [64]:
df = pd.read_csv('datasets/corn.csv')
df[df['YIELD'] == 0.00]

Index(['YEAR', 'PCODE', 'YIELD', 'MOIST', 'TW', 'DAYS'], dtype='object')

In [ ]:
# Making it fit the standard
df = pd.read_csv('datasets/corn.csv')
df.COUNTY = df.COUNTY.str.upper()
df.LOC = df.LOC.replace('SF', 'ST')
df = df[pd.notna(df['NAME']) & ~df['NAME'].astype(str).str.contains('LSD (0.05)')]
df = df[pd.notna(df['NAME']) & ~df['NAME'].astype(str).str.contains('LSD (0.05)*')]
# df.to_csv('datasets/corn.csv', index=False)

### Sorghum

In [15]:
df = pd.read_csv('datasets/sorghum.csv')
df

,YEAR,LOC,COUNTY,BRAND,NAME,PCODE,YIELD,WATER_REGIME,PAVG,MOIST,TW,DAYS
0,1983,THD,THOMAS,NaN,XXX,1086,82,Dryland,102,11,57,62
1,1994,STD,STAFFORD,NaN,15,1695,90,Dryland,106,18,52,65
2,1994,STD,STAFFORD,NaN,12,1775,98,Dryland,115,17,52,63
3,1994,RND,RENO,NaN,135,1858,67,Dryland,79,14,56,66
4,1994,RND,RENO,NaN,150,1861,65,Dryland,78,13,58,66
...,...,...,...,...,...,...,...,...,...,...,...,...
14063,2023,RND,RENO,DYNA-GRO,M72GB71,3038,112,Dryland,94,15,53,0
14064,2023,RND,RENO,DYNA-GRO,M63GB78,3074,135,Dryland,113,15,55,0
14065,2023,RND,RENO,DYNA-GRO,M67GB87,3076,112,Dryland,94,14,52,0
14066,2023,RND,RENO,RAGT,AC2104,3087,112,Dryland,94,11,53,0


In [167]:
# removing TANT in LOC
df = df[pd.notna(df['LOC']) & ~df['LOC'].astype(str).str.contains('TANT')]
# df.to_csv('datasets/sorghum.csv', index=False)

updating the pcodes, boy was it difficulty

In [168]:
pcode_df = pd.read_excel('sorghum_pcodes.xlsx')
pcode_df.PCODE = pcode_df.PCODE.astype('str')
pcode_df

,PCODE,BRAND,NAME
0,1,Pride of Saline,OP
1,2,DELTAPINE,G-777W
2,3,Westland Var,NaN
3,6,Public,RS 650
4,7,Midland Var,NaN
...,...,...,...
825,995,Pfizer,M 550G
826,996,Pfizer,M 568G
827,997,Paymaster,GR 1020
828,998,Paymaster,DR 1075


In [169]:
filtered_df = df[(df['YEAR'] == 1994) & (df['NAME'].isin(pcode_df['PCODE']))]
filtered_index = filtered_df.index
filtered_df

,YEAR,LOC,COUNTY,WATER_REGIME,BRAND,NAME,PCODE,YIELD,PAVG,MOIST,TW,DAYS
5,1994,RND,RENO,Dryland,NaN,151,1256,73,86,14,56,61
6,1994,STD,STAFFORD,Dryland,NaN,2,1822,73,86,16,57,57
7,1994,STD,STAFFORD,Dryland,NaN,7,1436,83,97,16,56,57
9,1994,STD,STAFFORD,Dryland,NaN,22,1826,93,109,17,52,54
12,1994,HVD,HARVEY,Dryland,NaN,79,1790,106,111,11,58,61
...,...,...,...,...,...,...,...,...,...,...,...,...
747,1994,BRD,BROWN,Dryland,NaN,33,1777,97,87,14,60,68
749,1994,BRD,BROWN,Dryland,NaN,16,1446,108,97,15,61,65
752,1994,BRD,BROWN,Dryland,NaN,148,1033,96,86,15,59,62
754,1994,BRD,BROWN,Dryland,NaN,147,1820,90,81,15,62,62


In [170]:
result_df = pd.merge(filtered_df, pcode_df, left_on='NAME', right_on='PCODE', how='left')
result_df = result_df.drop(['BRAND_x', 'NAME_x', 'PCODE_y'], axis=1)
result_df = result_df.rename(columns={'PCODE_x': 'PCODE', 'BRAND_y': 'BRAND', 'NAME_y': 'NAME'})
result_df = result_df[['YEAR', 'LOC', 'COUNTY', 'WATER_REGIME', 'BRAND', 'NAME', 'PCODE', 'YIELD', 'PAVG', 'MOIST', 'TW', 'DAYS']]
result_df

,YEAR,LOC,COUNTY,WATER_REGIME,BRAND,NAME,PCODE,YIELD,PAVG,MOIST,TW,DAYS
0,1994,RND,RENO,Dryland,Paymaster,Apache,1256,73,86,14,56,61
1,1994,STD,STAFFORD,Dryland,DELTAPINE,G-777W,1822,73,86,16,57,57
2,1994,STD,STAFFORD,Dryland,Midland Var,NaN,1436,83,97,16,56,57
3,1994,STD,STAFFORD,Dryland,Frontier,400C,1826,93,109,17,52,54
4,1994,HVD,HARVEY,Dryland,DEKALB,F-63,1790,106,111,11,58,61
...,...,...,...,...,...,...,...,...,...,...,...,...
238,1994,BRD,BROWN,Dryland,Public,RS 610,1777,97,87,14,60,68
239,1994,BRD,BROWN,Dryland,Martin Var,NaN,1446,108,97,15,61,65
240,1994,BRD,BROWN,Dryland,Paymaster,Kiowa,1033,96,86,15,59,62
241,1994,BRD,BROWN,Dryland,Paymaster,Comanche,1820,90,81,15,62,62


In [172]:
df.loc[filtered_index, :] = result_df.set_index(filtered_index)
df.head(10)

,YEAR,LOC,COUNTY,WATER_REGIME,BRAND,NAME,PCODE,YIELD,PAVG,MOIST,TW,DAYS
0,1983,THD,THOMAS,Dryland,NaN,XXX,1086,82,102,11,57,62
1,1994,STD,STAFFORD,Dryland,NaN,15,1695,90,106,18,52,65
2,1994,STD,STAFFORD,Dryland,NaN,12,1775,98,115,17,52,63
3,1994,RND,RENO,Dryland,NaN,135,1858,67,79,14,56,66
4,1994,RND,RENO,Dryland,NaN,150,1861,65,78,13,58,66
5,1994,RND,RENO,Dryland,Paymaster,Apache,1256,73,86,14,56,61
6,1994,STD,STAFFORD,Dryland,DELTAPINE,G-777W,1822,73,86,16,57,57
7,1994,STD,STAFFORD,Dryland,Midland Var,NaN,1436,83,97,16,56,57
8,1994,STD,STAFFORD,Dryland,NaN,10,1495,77,91,16,52,56
9,1994,STD,STAFFORD,Dryland,Frontier,400C,1826,93,109,17,52,54


In [173]:
df.to_csv('datasets/sorghum_2.csv', index=False)


should i do this on the earlier years on the column NAME? if so there are some that are still missing like 120
ans: Yes, ignore missing

### Sunflower

In [32]:
df = pd.read_csv('datasets/sunflower.csv')
df

,YEAR,LOC,COUNTY,BRAND,NAME,PCODE,YIELD,WATER_REGIME,DAYS
0,1998,GRDO,GREELEY,ASGROW SEED,AS5303,0,76,Dryland,57
1,1998,GRDO,GREELEY,INTERSTATE PAYCO SEED,IS 4049,0,74,Dryland,63
2,1998,GRDO,GREELEY,KAYSTAR SEED,9501,0,73,Dryland,63
3,1998,GRDO,GREELEY,CARGILL,X 3272,0,62,Dryland,58
4,1998,GRDO,GREELEY,MYCOGEN SEEDS,8488NS,0,76,Dryland,62
...,...,...,...,...,...,...,...,...,...
2341,2023,NaN,RENO,DYNA GRO,H47HO11EX,1013,1365,Dryland,0
2342,2023,NaN,RENO,DYNAGRO,H45NS16CL,1014,1726,Dryland,0
2343,2023,NaN,RENO,NUSEED,N4H490 E,1016,1756,Dryland,0
2344,2023,NaN,RENO,NUSEED,Hornet,920,2199,Dryland,0


In [33]:
df.LOC = df.LOC.str.upper()
df[df.YEAR == 2008].head()

,YEAR,LOC,COUNTY,BRAND,NAME,PCODE,YIELD,WATER_REGIME,DAYS
1284,2008,THIO,THOMAS,Monsanto,MH6640,0,99,Irrigated,58
1285,2008,THIO,THOMAS,Monsanto,MH6643,0,81,Irrigated,55
1286,2008,THIO,THOMAS,Monsanto,MH7633,0,91,Irrigated,57
1287,2008,THIO,THOMAS,Fontanelle,902NS,0,80,Irrigated,59
1288,2008,THIO,THOMAS,Garst,NX43489,0,105,Irrigated,57


In [25]:
# Fixing lower case LOC causing unknown locations
counties = pd.read_csv("datasets/county_data.csv")
c_dict = dict(zip(counties['ACRONYM'], counties['COUNTY']))
c_dict['FN'] = 'FINNEY'
c_dict['GR'] = 'GREELEY'
df = split_LOC(df, c_dict)
df[df.YEAR == 2008].head()
# df.to_csv('datasets/sunflower.csv')

In [34]:
# Splitting LOC last letter into column called TYPE: O - Oil; C - Confectionary
df['TYPE'] = df['LOC'].str[-1].map(
    {'O': 'Oil',
     'C': 'Confectionary',
     np.nan: 'Unknown'}
)
df.head(-10)

,YEAR,LOC,COUNTY,BRAND,NAME,PCODE,YIELD,WATER_REGIME,DAYS,TYPE
0,1998,GRDO,GREELEY,ASGROW SEED,AS5303,0,76,Dryland,57,Oil
1,1998,GRDO,GREELEY,INTERSTATE PAYCO SEED,IS 4049,0,74,Dryland,63,Oil
2,1998,GRDO,GREELEY,KAYSTAR SEED,9501,0,73,Dryland,63,Oil
3,1998,GRDO,GREELEY,CARGILL,X 3272,0,62,Dryland,58,Oil
4,1998,GRDO,GREELEY,MYCOGEN SEEDS,8488NS,0,76,Dryland,62,Oil
...,...,...,...,...,...,...,...,...,...,...
2331,2023,NaN,RENO,CROPLAN,CP 7919CL,970,1898,Dryland,0,Unknown
2332,2023,NaN,RENO,CROPLAN,CP 455E,951,1819,Dryland,0,Unknown
2333,2023,NaN,RENO,DYNA GRO,H42HO18CL,971,1660,Dryland,0,Unknown
2334,2023,NaN,RENO,DYNA GRO,H49HO19CL,972,2099,Dryland,0,Unknown


In [35]:
# df.to_csv('datasets/sunflower.csv', index=False)

### Wheat

In [2]:
df = pd.read_csv('datasets/wheat.csv')
df

,YEAR,LOC,BRAND,NAME,YIELD,TW,MOIST,HT,COUNTY,WATER_REGIME,HEAD
0,1994,ELDF,AgriPro,Longhorn,31,58,13,29,ELLIS,Dryland,139
1,1994,ELDF,----,Cimarron,33,62,11,26,ELLIS,Dryland,135
2,1994,ELDF,Star,Salute,37,59,13,26,ELLIS,Dryland,140
3,1994,ELDF,----,Karl,34,61,10,28,ELLIS,Dryland,134
4,1994,ELDF,----,Karl 92,34,61,11,27,ELLIS,Dryland,132
...,...,...,...,...,...,...,...,...,...,...,...
14196,2023,RPD,MERIDIAN,MS MAVERICK,26,60,0,0,REPUBLIC,Dryland,0
14197,2023,RPD,LIMAGRAIN,LCS JULEP,16,54,0,0,REPUBLIC,Dryland,0
14198,2023,RPD,LIMAGRAIN,LCS HELIX AX,19,59,0,0,REPUBLIC,Dryland,0
14199,2023,RPD,LIMAGRAIN,LCS ATOMIC AX,14,56,0,0,REPUBLIC,Dryland,0


In [4]:
# Removing entries with NC, EC, NWD, SWD, IRR, SC NE, SE in LOC
df = df[~df['LOC'].isin(['NC', 'EC', 'NWD', 'SWD', 'IRR', 'SC', 'NE', 'SE'])]
df
# df.to_csv('datasets/wheat.csv', index=False)


## Matching strings

In [29]:
from thefuzz import fuzz, process

In [44]:
df = pd.read_csv('datasets/wheat.csv')
unique_names = df['NAME'].fillna(0).unique()
unique_names = [str(item) for item in unique_names]
unique_names

['Longhorn',
 'Cimarron',
 'Salute',
 'Karl',
 'Karl 92',
 'Jagger',
 'KS92PO263-137 Exp',
 'KS92PO363-134 Exp',
 'Larned',
 'XH1529 Exp',
 'Voyager',
 'Laredo',
 'Hickock',
 'BNW3',
 'TAM 300',
 '7853',
 '21-193 Exp',
 'Discovery',
 'Tomahawk',
 'Ponderosa',
 'Scout 66',
 '(S) 2500 Exp',
 '(W) Rio Blanco',
 'Ogallala',
 'Newton',
 '(W) Arlin',
 'Vista',
 'Triumph 64',
 '9001',
 'BNW2',
 'BNW1',
 'Arapahoe',
 'TAM 200',
 'Ike',
 'TAM 107',
 'Colby',
 'XH1756 Exp',
 '2163',
 'Champ',
 'Tonkawa',
 '2180',
 'BC3',
 'BC2',
 '(S) Excel',
 '(S) MO12258 Exp',
 '(W) W88-2619W Exp',
 '(S) Jackson',
 'BNE1',
 'Custer',
 'BC4',
 '(S) Ernie (MO12256Ex',
 'W91-287 Exp',
 'W91-091 Exp',
 'Hickok',
 '(S) Boone',
 'BNE3',
 'HR 153',
 '(S) SR 204',
 '(S) SR 205',
 'Arkan',
 '(S) T441',
 '(S) Freedom',
 'NE89522 Exp',
 'BNE2',
 'Jagger-P',
 '(S) Caldwell',
 '(S) Cardinal',
 '(S) Clark',
 'KS91H153-2',
 '2172',
 'Alliance',
 'Yuma',
 'AP 7501',
 'WX92-3210 Exp',
 'BC1',
 '(S) 2500',
 'Colby 94',
 'Pecos'

In [61]:
for name in unique_names:
    match, score = process.extractOne(name, unique_names)
    # You can adjust the threshold score based on your requirements
    if score >= 80:
        if name != match:
            print(f'Match:{name, match}')
    else:
        print(f'Error:{name}')

Match:('Sturdy 2K', 'Sturdy-2K')
Match:('PostRock', 'Postrock')
Match:('(S) COKER 9553', '(S) Coker 9553')
Match:('DUSTER', 'Duster')
Match:('CENTERFIELD', 'Centerfield')
Match:('HITCH', 'Hitch')
Match:('OVERLEY', 'Overley')
Match:('FULLER', 'Fuller')
Match:('JAGGER', 'Jagger')
Match:('SANTA FE', 'Santa Fe')
Match:('POSTROCK', 'Postrock')
Match:('ART', 'Art')
Match:('ARMOUR', 'Armour')
Match:('ENDURANCE', 'Endurance')
Match:('BILLINGS', 'Billings')
Match:('EVEREST', 'Everest')
Match:('BILL BROWN', 'Bill Brown')
Match:('RIPPER', 'Ripper')
Match:('WINTERHAWK', 'Winterhawk')
Match:('(W) ASPEN', '(W) Aspen')
Match:('SHOCKER', 'Shocker')
Match:('OK BULLET', 'OK Bullet')
Match:('JAGALENE', 'Jagalene')
Match:('(W) RONL', '(W) RonL')
Match:('HATCHER', 'Hatcher')
Match:('(W) DANBY', '(W) Danby')
Match:('IKE', 'Ike')
Match:('(W) SNOWMASS', '(W) Snowmass')
Match:('(W) TIGER', '(W) Tiger')
Match:('JACKPOT', 'JackPot')
Match:('SY Wolf', 'SY WOLF')
Match:('Greer', 'GREER')
Match:('Robidoux', 'ROBIDO

example of typo:
DK647Bty pcode 3283
DK647BtY pcode 3283

example of two different genotypes that looks like a typo:
441Bt pcode 3880
441BT pcode 3743